In [1]:
import os

In [2]:
%pwd

'e:\\ETE_Disease_Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\ETE_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_dir

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])
    
    def get_callbacks_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_dir([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        callbacks_config = CallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return callbacks_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Callbacks:
    def __init__(self,config = CallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()
    callbacks = Callbacks(config=callbacks_config)
    callback_list = callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2024-05-27 22:52:50,612: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-27 22:52:50,620: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-27 22:52:50,623: INFO: common: created a directory at: artifacts]
[2024-05-27 22:52:50,629: INFO: common: created a directory at: artifacts\callbacks\chcekpoint_dir]
[2024-05-27 22:52:50,632: INFO: common: created a directory at: artifacts\callbacks\tensorboard_log_dir]
